# Juliaによる非線形方程式の全解探索アルゴリズムの実装

## Krawczyk法

解きたい方程式を$ f(x)=0, f:R^n \rightarrow R^n$とする。

解の存在を検証したい領域を$I \in R^n$とする。

cを$I$の中心とする。

$R$を$f'(c)$の近似逆行列とする。

$K(I):=c-Rf(c)+(R-Rf'(I)(I-c))$をKrawczyk写像という。

$K(I) \in int(I)$が成立するならば$f(x)=0$の真の解が$I$内に一意に存在する。


## 解の非存在条件
Krawczyk写像を利用し、解の非存在性を示すことができる。区間$X$に対し、$X \setminus K(X)$には解が存在しないので、
与えられた区間ベクトル$X \in IR^n $に対して、

$K(X) \cap X = \emptyset$

ならば、非線形方程式$f(X) = 0$の解は存在しない。

また、区間演算の性質より、以下のような非存在条件が考えられる。

与えられた区間ベクトル$X \in IR^n$に対して

$ 0 \notin f(X) $

または

$0 \notin f(c) + f'(X)(X-c) $

ならば、非線形方程式$f(X) = 0$の解は存在しない。

## Juliaによる区間演算

In [25]:
using IntervalArithmetic


X = (0.1 .. 0.1)
Y = (0.3 .. 0.4)

@show X
@show Y
@show X+Y
@show X-Y
@show X*Y
@show X/Y

X = Interval(0.09999999999999999, 0.1)
Y = Interval(0.3, 0.4)
X + Y = Interval(0.39999999999999997, 0.5000000000000001)
X - Y = Interval(-0.30000000000000004, -0.19999999999999998)
X * Y = Interval(0.029999999999999995, 0.04000000000000001)
X / Y = Interval(0.24999999999999994, 0.33333333333333337)


Interval(0.24999999999999994, 0.33333333333333337)

## Juliaによる線形代数演算

In [26]:
using LinearAlgebra

X = [1 2;3 4]
Y = [5 6;7 8]

@show typeof(X)
@show X+Y
@show X*Y
@show inv(X) 

typeof(X) = Matrix{Int64}
X + Y = [6 8; 10 12]
X * Y = [19 22; 43 50]
inv(X) = [-1.9999999999999996 0.9999999999999998; 1.4999999999999998 -0.4999999999999999]


2×2 Matrix{Float64}:
 -2.0   1.0
  1.5  -0.5

## Juliaによる自動微分

In [27]:
using ForwardDiff

f((x1,x2)) = x1^2-x2-1
g((x1,x2)) = (x1-2)^2 -x2-1
F((x,y))=[f((x,y));g((x,y))]
j1 = ForwardDiff.jacobian(F,[1. 1.])
j2 = ForwardDiff.jacobian(F,[(0. ..  1.) (0. .. 1.)])

@show j1
@show j2

j1 = [2.0 -1.0; -2.0 -1.0]
j2 = Interval{Float64}[Interval(0.0, 2.0) Interval(-1.0, -1.0); Interval(-4.0, -2.0) Interval(-1.0, -1.0)]


2×2 Matrix{Interval{Float64}}:
 Interval(0.0, 2.0)    Interval(-1.0, -1.0)
 Interval(-4.0, -2.0)  Interval(-1.0, -1.0)

## 全解探索アルゴリズムのソースコード

In [28]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = X*1
    for i = 1:size(X)[1]
        L[i]= intersect(X[i],Y[i])
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=0
    max_index=0
    min_diam=diam(X[1])
    min_index=1
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
        if diam(x)<min_diam
            min_diam=diam(x)
            min_index=i
        end
    end

    if diam(X[min_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end

function border(K,I)
    c=0.9
    K_width=diam.(K)
    I_width=diam.(I)
    max=0
    for i=1:size(K)[1]
        if (K_width[i]/I_width[i])>max
            max=K_width[i]/I_width[i]
        end
    end
    if max<=0.9
        return true
    else
        return false
    end
end

function remove_duplication(S)
    tmp_index=[]
    new_s=[]
    for i=1:size(S)[1]-1
        for j = i+1:size(S)[1]
            if !orig_is_empty(S[i],S[j])
                
                if (i in tmp_index) || (j in tmp_index)
                    push!(tmp_index,i)
                    push!(tmp_index,j)
                    continue
                end
                push!(tmp_index,i)
                push!(tmp_index,j)
                I = orig_intersect(S[i],S[j])
                push!(new_s,I)
            end
        end
    end
    for i=1:size(S)[1]
        if !(i in tmp_index)
            push!(new_s,S[i])
        end
    end
    
    return new_s
end


function allsol(F,X)
    dimention = size(X)[1]
    
    L = [X] #調査する区間のリスト
    S = Array[] #見つかった解のリスト
    
    while (size(L)[1] != 0)
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(X)) ) 
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(c)+j*(X-c)) )
            continue
        end
        
        R = []
        #逆行列計算
        try
            R = inv(ForwardDiff.jacobian(F,c))
        catch e
            #逆行列計算に失敗したら区間を分割
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch 
                continue
            end
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        #解なし
        if (orig_is_empty(K,X))
            continue 
        end
        
        #解あり
        if (orig_is_subset(K,X))　
            push!(S, K) #解のリストに追加
            continue
        end
        
        # 境界に乗ってた時
        if border(K,X)
            push!(L,K)
            continue
        end
        
        #分割して候補者区間に追加
        try
            X1,X2 = orig_devide(orig_intersect(K,X))
            #K and X を2分割
            push!(L, X1)
            push!(L, X2)
            continue
        catch
            continue
        end
    end
    
    #重複を排除
    S = remove_duplication(S)
    
    #区間を狭める
    for i =1:size(S)[1]
        si=S[i]
        while(maximum(radius,si) >= 10^-8)
            j=0
            try
                j = ForwardDiff.jacobian(F,si)
            catch
                break
            end
            c = mid.(si)
            R=0
            try 
                R = inv(ForwardDiff.jacobian(F,c)) 
            catch
                break
            end
            M = Matrix{Float64}(I,size(R))-R*j
            k = c -R*F(c)+M*(si-c)
            if orig_is_subset(k,si)
                si=k
            else
                break
            end
        end
        S[i]=si
    end
    
    setformat(:full)
#     for s=1:size(S)[1] 
#         print("解");print(s);print(": ");
#         for i in S[s]
#             print(i)
#             print(" ")
#         end
#         println("")
#     end
    
    return S
end

# 重解　＝＞区間幅に制限をつける
# 高速化？

allsol (generic function with 1 method)

## 検証1

In [29]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = 2*x1^3 + 2*x1*x2 - 22*x1+ x2^2 + 10 + pi
g((x1, x2)) = x1^2 + 2*x1*sin(x2) + 2*x2^3 - 14*x2 + 9.1

F1((x,y))=[f((x,y));g((x,y))]
runtime = @benchmark allsol(F1,X)
@show runtime

runtime = Trial(18.665 ms)


BenchmarkTools.Trial: 232 samples with 1 evaluation.
 Range (min … max):  18.665 ms … 35.473 ms  ┊ GC (min … max): 0.00% … 22.45%
 Time  (median):     19.389 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.643 ms ±  4.104 ms  ┊ GC (mean ± σ):  7.88% ± 10.99%

  █▆                                                           
  ███▅▄▄▄▃▃▄▃▂▁▃▁▃▂▂▁▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▂▄▅▄▃▃▂▃▃▃▃▃▁▁▂▁▁▁▁▂ ▃
  18.7 ms         Histogram: frequency by time          32 ms <

 Memory estimate: 9.88 MiB, allocs estimate: 230031.

### kvのallsolによる計算結果

[-4.1446150444051498,-4.1446150444051452]	[-3.28813192969558,-3.2881319296955777]

[0.82002197462817749,0.82002197462817839]	[-2.9339051715639717,-2.933905171563969]

[-3.4130997082062136,-3.4130997082062104]	[1.6985839371679012,1.6985839371679068]


[-3.4345149364856549,-3.4345149364856526]	[1.4044134733988069,1.4044134733988121]

[0.72030308132508302,0.7203030813250837]	[0.85337813273684248,0.85337813273684371]

[1.0843602587813725,1.0843602587813746]	[1.971798770579664,1.9717987705796661]

## 検証２

In [30]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1,x2)) = x1 + 2*x2 +1
g((x1,x2)) = x1^2 + 2*(x2^2) -1

F2((x,y))=[f((x,y));g((x,y))]
runtime = @benchmark allsol(F2,X)
@show runtime

runtime = Trial(2.504 ms)


BenchmarkTools.Trial: 1608 samples with 1 evaluation.
 Range (min … max):  2.504 ms … 18.302 ms  ┊ GC (min … max): 0.00% … 68.40%
 Time  (median):     2.605 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.101 ms ±  1.937 ms  ┊ GC (mean ± σ):  9.74% ± 12.23%

  █▅▄▂▁                                                       
  ██████▇▇▅▆▅▃▆▃▃▁▁▁▄▁▃▁▁▁▃▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇▆▆▄▅▅▄▅▄ █
  2.5 ms       Histogram: log(frequency) by time     12.9 ms <

 Memory estimate: 1.62 MiB, allocs estimate: 35959.

### kvのallsolによる計算結果

[0.3333333333333332,0.33333333333333349]	[-0.66666666666666686,-0.66666666666666651]

[-1.0000000000000005,-0.99999999999999977]	[-5.6222596042849915e-17,2.7044130804046339e-17]

## 検証3

In [31]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = x1^2 - x2^4

F3((x,y))=[f((x,y));g((x,y))]
runtime = @benchmark allsol(F3,X)
@show runtime

runtime = Trial(7.713 ms)


BenchmarkTools.Trial: 540 samples with 1 evaluation.
 Range (min … max):  7.713 ms … 32.695 ms  ┊ GC (min … max):  0.00% … 36.25%
 Time  (median):     7.889 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   9.271 ms ±  3.305 ms  ┊ GC (mean ± σ):  10.03% ± 14.15%

  █▅▂▁▁                                        ▂              
  ███████▇▅▇▅▄▅▁▁▁▅▁▄▁▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅██▇▆▄▅▅▅▄▁▄▄▅ ▆
  7.71 ms      Histogram: log(frequency) by time     19.4 ms <

 Memory estimate: 5.36 MiB, allocs estimate: 122114.

### kvのallsolによる計算結果

[-0.61803398874989513,-0.61803398874989456]	[-0.7861513777574235,-0.78615137775742305]

[-0.61803398874989513,-0.61803398874989456]	[0.78615137775742305,0.7861513777574235]

[0.61803398874989456,0.61803398874989513]	[-0.7861513777574235,-0.78615137775742305]

[0.61803398874989456,0.61803398874989513]	[0.78615137775742305,0.7861513777574235]

## 検証4(重解)

In [32]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = 2*x1^2 - x2 - 1

F4((x,y))=[f((x,y));g((x,y))]
runtime = @benchmark allsol(F4,X)
@show runtime

runtime = Trial(13.427 ms)


BenchmarkTools.Trial: 304 samples with 1 evaluation.
 Range (min … max):  13.427 ms … 31.654 ms  ┊ GC (min … max): 0.00% … 48.04%
 Time  (median):     14.295 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.433 ms ±  4.275 ms  ┊ GC (mean ± σ):  9.72% ± 13.17%

  █▇▆▄▁▂▂▃▁▂▁                    ▂ ▁▁                          
  ███████████▇▇▄▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄█████▄▄▁▆▇▄▆▆▄▁▁▁▁▄▄▄▁▁▄▄▁▁▆ ▇
  13.4 ms      Histogram: log(frequency) by time      30.3 ms <

 Memory estimate: 8.70 MiB, allocs estimate: 193299.

### kvのallsolによる計算結果

[-0.86602540378443882,-0.86602540378443848]	[0.49999999999999983,0.50000000000000023]

[0.86602540378443848,0.86602540378443882]	[0.49999999999999983,0.50000000000000023]


## 検証5 3次元以上

In [33]:
X = [(-1000. ..1000.),(-1000. .. 1000.),(-1000. .. 1000.)]

f((x1, x2, x3)) = 2*x1 + x2^2 + x3
g((x1, x2, x3)) = 2*x1^2 - x2 - x3
h((x1, x2, x3)) = x1 + x2 + x3

F5((x,y,z))=[f((x,y,z));g((x,y,z));h((x,y,z))]

runtime = @benchmark allsol(F5,X)
@show runtime

runtime = Trial(11.484 ms)


BenchmarkTools.Trial: 348 samples with 1 evaluation.
 Range (min … max):  11.484 ms … 29.087 ms  ┊ GC (min … max):  0.00% … 46.82%
 Time  (median):     12.582 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   14.369 ms ±  4.120 ms  ┊ GC (mean ± σ):  10.34% ± 14.23%

  ██▆▅▅▅▄ ▁                      ▁▁▁                           
  ███████▇██▆▄▆▇▆▆▄▁▄▁▄▁▄▄▁▄▁▄▁▄▇███▇▇▇▇▆▄▄▆▆▄▁▆▄▁▁▆▄▁▁▁▄▁▁▁▇ ▇
  11.5 ms      Histogram: log(frequency) by time      28.4 ms <

 Memory estimate: 7.74 MiB, allocs estimate: 158976.

[-4.9406564584124655e-324,4.9406564584124655e-324]
[-9.8813129168249309e-324,9.8813129168249309e-324]
[-9.8813129168249309e-324,9.8813129168249309e-324]

[-1.9075222407249442e-16,3.1292380856700906e-17]
[0.99999999999999966,1.0000000000000007]
[-1.0000000000000005,-0.99999999999999977]

[-0.50000000000000023,-0.49999999999999994]
[-0.36602540378443877,-0.36602540378443854]
[0.86602540378443848,0.86602540378443882]

[-0.50000000000000023,-0.49999999999999994]
[1.3660254037844383,1.3660254037844391]
[-0.86602540378443893,-0.86602540378443848]


## 今後

- 高速化
- KVのallsol_simpleとの速度比較